### Data Analysis

- **How many rows**: Number of roows in the dataset.
- **How many columns**: Number of columns in the dataset.
- **List of the columns**: List of columns in the dataset.
- **Total Products**: Number of products in the dataset.
- **Distinct Products**: Number of unique products.
- **Total Transactions**: Number of transactions recorded.
- **Total Customers**: Number of customers in the dataset.
- **Distinct Categories**: Number of unique product categories.


In [2]:

import pandas as pd
import os

path = r'C:\Users\moham\Apriori_VS_Word2Vec'
excel_file = 'df_merged_items_category.xlsx'
excel_file_path = os.path.join(path, excel_file)

# Load the dataset
df = pd.read_excel(excel_file_path)

# Basic Dataset Statistics
print("===== DATASET OVERVIEW =====")
print(f"How many rows: {df.shape[0]}")
print(f"How many columns: {df.shape[1]}")
print(f"List of columns: {list(df.columns)}")

# Total and distinct products
total_products = df.shape[0]
distinct_products = df['Itemname'].nunique()
print(f"Total Products: {total_products}")
print(f"Distinct Products: {distinct_products}")

# Transactions analysis
total_transactions = df['BillNo'].nunique()
print(f"Total Transactions: {total_transactions}")

# Customer analysis
total_customers = df['CustomerID'].nunique()
print(f"Total Customers: {total_customers}")

# Category analysis
if 'category' in df.columns:
    distinct_categories = df['category'].nunique()
    print(f"Distinct Categories: {distinct_categories}")
    # Category distribution
    category_counts = df['category'].value_counts()
    print("\n===== CATEGORY DISTRIBUTION =====")
    for category, count in category_counts.items():
        percentage = (count / total_products) * 100
        print(f"{category}: {count} products ({percentage:.2f}%)")

# Date range
if 'Date' in df.columns:
    min_date = df['Date'].min()
    max_date = df['Date'].max()
    print(f"\nDate Range: {min_date} to {max_date}")




===== DATASET OVERVIEW =====
How many rows: 520609
How many columns: 8
List of columns: ['BillNo', 'Itemname', 'Quantity', 'Date', 'Price', 'CustomerID', 'Country', 'category']
Total Products: 520609
Distinct Products: 4185
Total Transactions: 20208
Total Customers: 4297
Distinct Categories: 21

===== CATEGORY DISTRIBUTION =====
Kitchen & Dining: 105749 products (20.31%)
Home Decor: 97277 products (18.69%)
Stationery & Office: 47941 products (9.21%)
Seasonal & Holidays: 40666 products (7.81%)
Kids & Toys: 40642 products (7.81%)
Textiles & Cozy Items: 25770 products (4.95%)
Vintage & Collectibles: 24355 products (4.68%)
Fashion & Accessories: 24224 products (4.65%)
Party Supplies: 23609 products (4.53%)
Storage & Organization: 22690 products (4.36%)
Garden & Outdoor: 18558 products (3.56%)
Crafts & DIY: 17090 products (3.28%)
Lighting: 7022 products (1.35%)
Games & Puzzles: 7013 products (1.35%)
Electronics & Gadgets: 5269 products (1.01%)
Bath & Body: 5024 products (0.97%)
Gifts & Spec

### Transaction Patterns

Analyze the purchasing patterns visible in the data:

- **Transaction frequency over time**: Daily, weekly, and monthly patterns.
- **Average items per transaction**.
- **Distribution of transaction sizes**: Histogram showing the number of items per invoice.
- **Temporal patterns**: Time of day, day of week, and seasonal trends.

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import numpy as np
import os

# Path setup
path = r'C:\Users\moham\Apriori_VS_Word2Vec'
excel_file = 'df_merged_items_category.xlsx'
excel_file_path = os.path.join(path, excel_file)

# Load the dataset
df = pd.read_excel(excel_file_path)

# Ensure Date is in datetime format
if 'Date' in df.columns:
    if not pd.api.types.is_datetime64_any_dtype(df['Date']):
        df['Date'] = pd.to_datetime(df['Date'])

# Create plot directory
plots_dir = os.path.join(path, 'analysis_plots')
if not os.path.exists(plots_dir):
    os.makedirs(plots_dir)

# ===== TRANSACTION PATTERNS ANALYSIS =====
print("\n===== TRANSACTION PATTERNS =====")

# 1. Transaction frequency over time
print("\n1. Transaction Frequency Over Time")

# Add date components
df['Date_Only'] = df['Date'].dt.date
df['Day'] = df['Date'].dt.day_name()
df['Hour'] = df['Date'].dt.hour
df['Month'] = df['Date'].dt.month_name()
df['Week'] = df['Date'].dt.isocalendar().week

# Daily transaction counts
daily_transactions = df.groupby('Date_Only')['BillNo'].nunique()
print("\nDaily Transaction Counts:")
print(daily_transactions)

# Plot daily transactions
plt.figure(figsize=(12, 6))
daily_transactions.plot()
plt.title('Number of Transactions by Day')
plt.xlabel('Date')
plt.ylabel('Number of Transactions')
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, 'daily_transactions.png'))
plt.close()

# Day of week analysis
day_of_week_transactions = df.groupby('Day')['BillNo'].nunique().reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
print("\nTransactions by Day of Week:")
print(day_of_week_transactions)

# Plot day of week transactions
plt.figure(figsize=(10, 6))
day_of_week_transactions.plot(kind='bar')
plt.title('Number of Transactions by Day of Week')
plt.xlabel('Day of Week')
plt.ylabel('Number of Transactions')
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, 'day_of_week_transactions.png'))
plt.close()

# Hourly pattern
hourly_transactions = df.groupby('Hour')['BillNo'].nunique()
print("\nTransactions by Hour of Day:")
print(hourly_transactions)

# Plot hourly transactions
plt.figure(figsize=(12, 6))
hourly_transactions.plot(kind='bar')
plt.title('Number of Transactions by Hour of Day')
plt.xlabel('Hour')
plt.ylabel('Number of Transactions')
plt.xticks(range(24), [f'{i}:00' for i in range(24)])
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, 'hourly_transactions.png'))
plt.close()

# 2. Average items per transaction
print("\n2. Average Items per Transaction")
# Count items per transaction
items_per_transaction = df.groupby('BillNo').size()
avg_items = items_per_transaction.mean()
median_items = items_per_transaction.median()
min_items = items_per_transaction.min()
max_items = items_per_transaction.max()

print(f"Average Items per Transaction: {avg_items:.2f}")
print(f"Median Items per Transaction: {median_items}")
print(f"Min Items per Transaction: {min_items}")
print(f"Max Items per Transaction: {max_items}")

# 3. Distribution of transaction sizes
print("\n3. Distribution of Transaction Sizes")
# Plot histogram of items per transaction
plt.figure(figsize=(12, 6))
plt.hist(items_per_transaction, bins=30)
plt.title('Distribution of Items per Transaction')
plt.xlabel('Number of Items')
plt.ylabel('Number of Transactions')
plt.grid(axis='y', alpha=0.75)
plt.savefig(os.path.join(plots_dir, 'transaction_size_histogram.png'))
plt.close()

# Calculate percentiles
percentiles = np.percentile(items_per_transaction, [25, 50, 75, 90, 95, 99])
print("\nTransaction Size Percentiles:")
print(f"25th percentile: {percentiles[0]:.1f} items")
print(f"50th percentile (median): {percentiles[1]:.1f} items")
print(f"75th percentile: {percentiles[2]:.1f} items")
print(f"90th percentile: {percentiles[3]:.1f} items")
print(f"95th percentile: {percentiles[4]:.1f} items")
print(f"99th percentile: {percentiles[5]:.1f} items")

# Transaction size distribution (in bins)
size_bins = [1, 5, 10, 15, 20, 30, 50, 100, np.inf]
size_labels = ['1-4', '5-9', '10-14', '15-19', '20-29', '30-49', '50-99', '100+']
transaction_size_binned = pd.cut(items_per_transaction, bins=size_bins, labels=size_labels)
size_counts = transaction_size_binned.value_counts().sort_index()

print("\nTransaction Size Distribution:")
total_transactions = len(items_per_transaction)
for size, count in size_counts.items():
    percentage = (count / total_transactions) * 100
    print(f"{size} items: {count} transactions ({percentage:.2f}%)")

# Plot binned transaction sizes
plt.figure(figsize=(10, 6))
size_counts.plot(kind='bar')
plt.title('Transaction Size Distribution')
plt.xlabel('Number of Items')
plt.ylabel('Number of Transactions')
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, 'transaction_size_bins.png'))
plt.close()

# 4. Temporal patterns - heatmap showing day of week vs hour
print("\n4. Temporal Patterns - Day of Week vs Hour")
# Create hour-of-day vs day-of-week heatmap
day_hour_counts = df.groupby(['Day', 'Hour'])['BillNo'].nunique().unstack()
# Reorder days
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_hour_counts = day_hour_counts.reindex(day_order)

plt.figure(figsize=(14, 8))
sns.heatmap(day_hour_counts, cmap='YlGnBu', annot=False)
plt.title('Number of Transactions by Day of Week and Hour')
plt.xlabel('Hour of Day')
plt.ylabel('Day of Week')
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, 'day_hour_heatmap.png'))
plt.close()

# Most common hour for each day
busiest_hours = day_hour_counts.idxmax(axis=1)
print("\nBusiest Hour for Each Day:")
for day, hour in busiest_hours.items():
    print(f"{day}: {hour}:00")

print(f"\nVisualizations saved to {plots_dir}")
print("\nTransaction pattern analysis complete.")


===== TRANSACTION PATTERNS =====

1. Transaction Frequency Over Time

Daily Transaction Counts:
Date_Only
2010-12-01    125
2010-12-02    141
2010-12-03     68
2010-12-05     88
2010-12-06    102
             ... 
2011-12-05    129
2011-12-06    117
2011-12-07    117
2011-12-08    122
2011-12-09     44
Name: BillNo, Length: 305, dtype: int64

Transactions by Day of Week:
Day
Monday       3159.0
Tuesday      3615.0
Wednesday    3751.0
Thursday     4292.0
Friday       3209.0
Saturday        NaN
Sunday       2182.0
Name: BillNo, dtype: float64

Transactions by Hour of Day:
Hour
6        1
7       26
8      534
9     1485
10    2352
11    2445
12    3240
13    2774
14    2472
15    2394
16    1386
17     714
18     227
19     141
20      18
Name: BillNo, dtype: int64

2. Average Items per Transaction
Average Items per Transaction: 25.76
Median Items per Transaction: 15.0
Min Items per Transaction: 1
Max Items per Transaction: 1114

3. Distribution of Transaction Sizes

Transaction Size Pe

C:\Users\moham\AppData\Local\Temp\ipykernel_29388\3219655145.py:161: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  busiest_hours = day_hour_counts.idxmax(axis=1)


### Product Analysis

Examine the product-related characteristics:

- **Top 10-20 most frequently purchased products**.
- **Product category distribution**: Percentage of items in each category.
- **Price distribution**: Across different product categories.
- **Product popularity vs. price**: Correlation analysis.

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from scipy.stats import pearsonr

# Path setup
path = r'C:\Users\moham\Apriori_VS_Word2Vec'
excel_file = 'df_merged_items_category.xlsx'
excel_file_path = os.path.join(path, excel_file)

# Load the dataset
df = pd.read_excel(excel_file_path)

# Create plot directory if it doesn't exist
plots_dir = os.path.join(path, 'analysis_plots')
if not os.path.exists(plots_dir):
    os.makedirs(plots_dir)

# ===== PRODUCT ANALYSIS =====
print("\n===== PRODUCT ANALYSIS =====")

# 1. Top products by frequency
print("\n1. Top Most Frequently Purchased Products")
product_counts = df['Itemname'].value_counts()
top_20_products = product_counts.head(20)

print("\nTop 20 Most Frequently Purchased Products:")
for i, (product, count) in enumerate(top_20_products.items(), 1):
    percentage = (count / len(df)) * 100
    print(f"{i}. {product}: {count} purchases ({percentage:.2f}%)")

# Plot top 20 products
plt.figure(figsize=(14, 10))
top_20_products.plot(kind='barh')
plt.title('Top 20 Most Frequently Purchased Products')
plt.xlabel('Number of Purchases')
plt.ylabel('Product Name')
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, 'top_20_products.png'))
plt.close()

# 2. Product category distribution
if 'category' in df.columns:
    print("\n2. Product Category Distribution")
    category_counts = df['category'].value_counts()
    
    print("\nProduct Category Distribution:")
    for category, count in category_counts.items():
        percentage = (count / len(df)) * 100
        print(f"{category}: {count} products ({percentage:.2f}%)")
    
    # Plot category distribution
    plt.figure(figsize=(12, 8))
    category_counts.plot(kind='bar')
    plt.title('Product Category Distribution')
    plt.xlabel('Category')
    plt.ylabel('Number of Products')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(os.path.join(plots_dir, 'category_distribution_bar.png'))
    plt.close()
    
    # Pie chart for categories
    plt.figure(figsize=(10, 10))
    plt.pie(category_counts, labels=category_counts.index, autopct='%1.1f%%')
    plt.title('Product Category Distribution')
    plt.axis('equal')
    plt.tight_layout()
    plt.savefig(os.path.join(plots_dir, 'category_distribution_pie.png'))
    plt.close()

# 3. Price distribution across categories
if 'Price' in df.columns and 'category' in df.columns:
    print("\n3. Price Distribution Across Categories")
    
    # Basic price statistics by category
    price_stats = df.groupby('category')['Price'].agg(['min', 'max', 'mean', 'median', 'std'])
    price_stats = price_stats.sort_values('mean', ascending=False)
    
    print("\nPrice Statistics by Category:")
    for category, stats in price_stats.iterrows():
        print(f"{category}:")
        print(f"  Min: £{stats['min']:.2f}")
        print(f"  Max: £{stats['max']:.2f}")
        print(f"  Mean: £{stats['mean']:.2f}")
        print(f"  Median: £{stats['median']:.2f}")
        print(f"  Std Dev: £{stats['std']:.2f}")
    
    # Box plot of price by category
    plt.figure(figsize=(14, 8))
    sns.boxplot(x='category', y='Price', data=df)
    plt.title('Price Distribution by Category')
    plt.xlabel('Category')
    plt.ylabel('Price (£)')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(os.path.join(plots_dir, 'price_distribution_boxplot.png'))
    plt.close()
    
    # Violin plot for more detailed distribution
    plt.figure(figsize=(14, 8))
    sns.violinplot(x='category', y='Price', data=df, cut=0)
    plt.title('Price Distribution by Category (Violin Plot)')
    plt.xlabel('Category')
    plt.ylabel('Price (£)')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(os.path.join(plots_dir, 'price_distribution_violin.png'))
    plt.close()

# 4. Product popularity vs. price correlation
if 'Price' in df.columns and 'Itemname' in df.columns:
    print("\n4. Product Popularity vs. Price Correlation")
    
    # Get product popularity and average price
    product_data = df.groupby('Itemname').agg({
        'Itemname': 'count',
        'Price': 'mean'
    }).rename(columns={'Itemname': 'Frequency'})
    
    # Calculate correlation
    correlation, p_value = pearsonr(product_data['Frequency'], product_data['Price'])
    print(f"\nCorrelation between product popularity and price: {correlation:.4f}")
    print(f"P-value: {p_value:.4f}")
    
    if p_value < 0.05:
        significance = "statistically significant"
    else:
        significance = "not statistically significant"
    
    print(f"The correlation is {significance} (p < 0.05).")
    
    if correlation < -0.5:
        interpretation = "strong negative"
    elif correlation < -0.3:
        interpretation = "moderate negative"
    elif correlation < -0.1:
        interpretation = "weak negative"
    elif correlation < 0.1:
        interpretation = "negligible"
    elif correlation < 0.3:
        interpretation = "weak positive"
    elif correlation < 0.5:
        interpretation = "moderate positive"
    else:
        interpretation = "strong positive"
    
    print(f"This represents a {interpretation} correlation.")
    
    # Scatter plot of price vs. popularity
    plt.figure(figsize=(12, 8))
    plt.scatter(product_data['Price'], product_data['Frequency'], alpha=0.5)
    plt.title('Product Popularity vs. Price')
    plt.xlabel('Average Price (£)')
    plt.ylabel('Number of Purchases')
    plt.grid(True, alpha=0.3)
    
    # Add trend line
    z = np.polyfit(product_data['Price'], product_data['Frequency'], 1)
    p = np.poly1d(z)
    plt.plot(product_data['Price'], p(product_data['Price']), "r--", 
             label=f"Trend line (y={z[0]:.2f}x+{z[1]:.2f})")
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(plots_dir, 'popularity_vs_price.png'))
    plt.close()
    
    # Log-scale scatter plot (often better for this type of data)
    plt.figure(figsize=(12, 8))
    plt.scatter(product_data['Price'], product_data['Frequency'], alpha=0.5)
    plt.title('Product Popularity vs. Price (Log Scale)')
    plt.xlabel('Average Price (£)')
    plt.ylabel('Number of Purchases (Log Scale)')
    plt.yscale('log')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(plots_dir, 'popularity_vs_price_log.png'))
    plt.close()
    
    # Top 10 most popular products with their prices
    top_10_with_price = product_data.sort_values('Frequency', ascending=False).head(10)
    print("\nTop 10 Most Popular Products with Average Prices:")
    for product, data in top_10_with_price.iterrows():
        print(f"{product}: {data['Frequency']} purchases, Average Price: £{data['Price']:.2f}")
    
    # Top 10 most expensive products with their popularity
    top_10_expensive = product_data.sort_values('Price', ascending=False).head(10)
    print("\nTop 10 Most Expensive Products with Popularity:")
    for product, data in top_10_expensive.iterrows():
        print(f"{product}: £{data['Price']:.2f}, {data['Frequency']} purchases")

print(f"\nVisualizations saved to {plots_dir}")
print("\nProduct analysis complete.")


===== PRODUCT ANALYSIS =====

1. Top Most Frequently Purchased Products

Top 20 Most Frequently Purchased Products:
1. WHITE HANGING HEART T-LIGHT HOLDER: 2269 purchases (0.44%)
2. JUMBO BAG RED RETROSPOT: 2087 purchases (0.40%)
3. REGENCY CAKESTAND 3 TIER: 1930 purchases (0.37%)
4. PARTY BUNTING: 1677 purchases (0.32%)
5. LUNCH BAG RED RETROSPOT: 1570 purchases (0.30%)
6. ASSORTED COLOUR BIRD ORNAMENT: 1465 purchases (0.28%)
7. SET OF 3 CAKE TINS PANTRY DESIGN: 1360 purchases (0.26%)
8. PACK OF 72 RETROSPOT CAKE CASES: 1328 purchases (0.26%)
9. LUNCH BAG  BLACK SKULL.: 1315 purchases (0.25%)
10. NATURAL SLATE HEART CHALKBOARD: 1246 purchases (0.24%)
11. JUMBO BAG PINK POLKADOT: 1231 purchases (0.24%)
12. HEART OF WICKER SMALL: 1206 purchases (0.23%)
13. JUMBO STORAGE BAG SUKI: 1191 purchases (0.23%)
14. PAPER CHAIN KIT 50'S CHRISTMAS: 1183 purchases (0.23%)
15. JUMBO SHOPPER VINTAGE RED PAISLEY: 1181 purchases (0.23%)
16. LUNCH BAG SPACEBOY DESIGN: 1169 purchases (0.22%)
17. LUNCH BA